# Papago RTT dataset 고유명사 살아 있는 것만 선택하기

In [2]:
import pandas as pd

from konlpy.tag import Mecab
from pororo import Pororo
from tqdm.auto import tqdm

from datasets import load_from_disk

[Korean Sentence Splitter]: Initializing Kss...


In [3]:
train = load_from_disk('/opt/ml/data/train_dataset/train')
ppg = pd.read_csv('./trainset_rtt_papago.csv')
mecab = Mecab()

In [4]:
len(ppg)

3952

In [5]:
# 조사 및 명사 띄워주기 위해서 -> 고유명사 판별 원활히 하기 위해서!
def tokenize(text):
    return mecab.morphs(text)

In [6]:
org_questions = list(map(tokenize, train['question'])) #mecab 적용

In [7]:
org_questions[0]

['대통령',
 '을',
 '포함',
 '한',
 '미국',
 '의',
 '행정부',
 '견제',
 '권',
 '을',
 '갖',
 '는',
 '국가',
 '기관',
 '은',
 '?']

In [8]:
new_questions = list(map(tokenize, ppg['question'])) #mecab 적용

In [9]:
new_questions[0]

['대통령',
 '을',
 '포함',
 '한',
 '미국',
 '의',
 '행정부',
 '를',
 '견제',
 '할',
 '수',
 '있',
 '는',
 '권한',
 '은',
 '어느',
 '국가',
 '기관',
 '에',
 '있',
 '습니까',
 '?']

macab으로 자른 것 -> 띄어쓰기를 중간에 넣어 한 문장으로 만들기

In [10]:
org_question=[]
for item in org_questions:
    k=''
    for i in item:
        k+=i+" "
    org_question.append(k.strip())

In [11]:
org_question[0]

'대통령 을 포함 한 미국 의 행정부 견제 권 을 갖 는 국가 기관 은 ?'

In [12]:
new_question = []
for item in new_questions:
    k=''
    for i in item:
        k+=i+" "
    new_question.append(k.strip())

In [13]:
new_question[0]

'대통령 을 포함 한 미국 의 행정부 를 견제 할 수 있 는 권한 은 어느 국가 기관 에 있 습니까 ?'

In [14]:
ner = Pororo(task="ner", lang="ko")

In [15]:
print(ner(org_question[0])) # 'O'판별한거 제외하고 나머지가 살아있는지 확인해보기!

[('대통령', 'CIVILIZATION'), (' ', 'O'), ('을', 'O'), (' ', 'O'), ('포함', 'O'), (' ', 'O'), ('한', 'O'), (' ', 'O'), ('미국', 'COUNTRY'), (' ', 'O'), ('의', 'O'), (' ', 'O'), ('행정부', 'ORGANIZATION'), (' ', 'O'), ('견제', 'O'), (' ', 'O'), ('권', 'O'), (' ', 'O'), ('을', 'O'), (' ', 'O'), ('갖', 'O'), (' ', 'O'), ('는', 'O'), (' ', 'O'), ('국가', 'O'), (' ', 'O'), ('기관', 'O'), (' ', 'O'), ('은', 'O'), (' ', 'O'), ('?', 'O')]


In [16]:
print(ner(new_question[0]))

[('대통령', 'CIVILIZATION'), (' ', 'O'), ('을', 'O'), (' ', 'O'), ('포함', 'O'), (' ', 'O'), ('한', 'O'), (' ', 'O'), ('미국', 'COUNTRY'), (' ', 'O'), ('의', 'O'), (' ', 'O'), ('행정부', 'ORGANIZATION'), (' ', 'O'), ('를', 'O'), (' ', 'O'), ('견제', 'O'), (' ', 'O'), ('할', 'O'), (' ', 'O'), ('수', 'O'), (' ', 'O'), ('있', 'O'), (' ', 'O'), ('는', 'O'), (' ', 'O'), ('권한', 'O'), (' ', 'O'), ('은', 'O'), (' ', 'O'), ('어느', 'O'), (' ', 'O'), ('국가', 'O'), (' ', 'O'), ('기관', 'O'), (' ', 'O'), ('에', 'O'), (' ', 'O'), ('있', 'O'), (' ', 'O'), ('습니까', 'O'), (' ', 'O'), ('?', 'O')]


In [20]:
matching_index = []
for idx in tqdm(range(len(org_question))):
    org = ner(org_question[idx])
    new = ner(new_question[idx])
    
    org_word=[org_describe[0] for org_describe in org if org_describe[1] !="O"]
    new_word=[new_describe[0] for new_describe in new if new_describe[1] !="O"]

    #고유명사 개수가 다르면
    if len(org_word) > len(new_word):
        continue
            
    ## 새로 만든 question에 기존 question의 고유 명사가 다 들어가 있어야한다!
    
    for i, orgword in enumerate(org_word):
        if orgword not in new_word:
            break
        if i == len(org_word)-1:
            matching_index.append(idx)

  0%|          | 0/3952 [00:00<?, ?it/s]

In [21]:
len(matching_index)

1682

In [22]:
ppg_ner = ppg.iloc[matching_index]

In [23]:
ppg_ner.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부를 견제할 수 있는 권한은 어느 국가기관에 있습니까?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제는 1717년에 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
6,호반,"제27회와 제73회에 출현한다. 관우는 자신을 저지하는 공수, 맹탄, 한복, 변희를...",누가 관우를 불태워 죽이려 했습니까?,mrc-0-000532,"{'answer_start': [68], 'text': ['형양태수 왕식']}",24395,360
12,이스트앵글리아 왕국,"웨하가 동앵글리아의 초대 왕으로 기록되고, 그 뒤를 우파가 뒤를 잇던, 동앵글리아 ...",우핑가스 왕조는 어디서 왔을까?,mrc-0-000425,"{'answer_start': [86], 'text': ['반신화적인 인물인 우파']}",19308,286
13,장면,장면은 귀국전 국무총리에 임명되었으나 이듬해인 1951년 2월에 귀국한 후 정식으로...,당신은 언제 한국으로 돌아 오셨나요?,mrc-0-001373,"{'answer_start': [26], 'text': ['1951년']}",6930,899


In [24]:
ppg_ner = ppg_ner.reset_index(drop=True)
ppg_ner

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부를 견제할 수 있는 권한은 어느 국가기관에 있습니까?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제는 1717년에 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
2,호반,"제27회와 제73회에 출현한다. 관우는 자신을 저지하는 공수, 맹탄, 한복, 변희를...",누가 관우를 불태워 죽이려 했습니까?,mrc-0-000532,"{'answer_start': [68], 'text': ['형양태수 왕식']}",24395,360
3,이스트앵글리아 왕국,"웨하가 동앵글리아의 초대 왕으로 기록되고, 그 뒤를 우파가 뒤를 잇던, 동앵글리아 ...",우핑가스 왕조는 어디서 왔을까?,mrc-0-000425,"{'answer_start': [86], 'text': ['반신화적인 인물인 우파']}",19308,286
4,장면,장면은 귀국전 국무총리에 임명되었으나 이듬해인 1951년 2월에 귀국한 후 정식으로...,당신은 언제 한국으로 돌아 오셨나요?,mrc-0-001373,"{'answer_start': [26], 'text': ['1951년']}",6930,899
...,...,...,...,...,...,...,...
1677,미국 알래스카 육군,"1940년, 제2차 세계 대전에 대비하기 위해 알래스카 방위군이 창설되었다. 랜드리...",알래스카 고속도로는 미국 본토에서 어떤 역할을 하나요?,mrc-1-000818,"{'answer_start': [295], 'text': ['병력과 물류를 보급할 ...",35815,529
1678,이오 (위성),이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...,무엇이 대부분의 이오 산의 형성에 영향을 미쳤나요?,mrc-0-005285,"{'answer_start': [317], 'text': ['지질 구조']}",8275,3445
1679,내니 다이어리,애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...,애니는 어디서 태어났니?,mrc-1-000699,"{'answer_start': [616], 'text': ['뉴저지']}",37764,450
1680,빈 필하모니 관현악단,1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,1854년 니콜라이의 뒤를 이어 상임 지휘자가 누가 되었나요?,mrc-0-003429,"{'answer_start': [255], 'text': ['칼 에케르트']}",12730,2252


In [25]:
ppg_ner.to_csv('papago.csv')

# Dataset 합치기

In [25]:
import pandas as pd
from datasets import load_from_disk

train = load_from_disk('/opt/ml/data/train_dataset/')
ppg = pd.read_csv('./trainset_rtt_papago.csv')

In [ ]:
print(dir)

In [26]:
ppg.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부를 견제할 수 있는 권한은 어느 국가기관에 있습니까?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대 인사 조직 관리의 출발점은 무엇이었나요?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제는 1717년에 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11세기에서 12세기에 제작된 본존불은 보통 어느 나라의 특징을 전파하나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,비문이 새겨진 유물의 그릇 수는 총 몇 개인가요?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


In [30]:
raw_df = train['train'].to_pandas()

In [31]:
raw_df.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


In [32]:
ppg['answers'] = ppg['answers'].map(eval)

In [41]:
train_dataset = pd.concat([raw_df, ppg]).reset_index(drop=True)

In [42]:
len(train_dataset)

7904

In [43]:
train_data = Dataset.from_pandas(train_dataset)

In [44]:
train_data

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 7904
})

In [11]:
# d1 = Dataset.from_dict({"a": [0, 1, 2]})
from datasets import Dataset, interleave_datasets

ppg_dataset = Dataset.from_dict({'title' : ppg['title'].to_list(),
                                 'context' : ppg['context'].to_list(),
                                 'question' : ppg['question'].to_list(),
                                 'id' : ppg['id'].to_list(),
                                 'answers' : ppg['answers'].to_list(),
                                 'document_id' : ppg['document_id'].to_list(),
                                 '__index_level_0__' : ppg['__index_level_0__'].to_list()
                                 })

In [12]:
ppg_dataset

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 3952
})

In [13]:
train['train']

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 3952
})

In [21]:
ppg_dataset.features

{'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'answers': Value(dtype='string', id=None),
 'document_id': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [8]:
from datasets import concatenate_datasets

In [24]:
train['train'].features

{'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'answers': {'answer_start': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
  'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)},
 'document_id': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [22]:
ppg_dataset_cast = ppg_dataset.cast(train['train'].features)

  0%|          | 0/1 [00:00<?, ?ba/s]

ArrowNotImplementedError: Unsupported cast from string to struct using function cast_struct

In [14]:
dataset = concatenate_datasets([train['train'], ppg_dataset])

ValueError: Features must match for all datasets

In [2]:
ppg['id']

0       mrc-1-000067
1       mrc-0-004397
2       mrc-1-000362
3       mrc-0-001510
4       mrc-0-000823
            ...     
3947    mrc-0-005285
3948    mrc-1-000699
3949    mrc-0-003429
3950    mrc-0-003956
3951    mrc-0-003589
Name: id, Length: 3952, dtype: object

In [3]:
type(train['train'])

datasets.arrow_dataset.Dataset

In [10]:
train

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [4]:
train['train']

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 3952
})

In [30]:
temp_id = train['train']['id'][:]
len(temp_id)

3952

In [31]:
temp_id.extend(ppg['id'].to_list())
len(temp_id)

7904

In [11]:
temp_q = train['train']['question'][:]
temp_q.extend(ppg['question'].to_list())

In [12]:
len(temp_q)

7904

In [13]:
temp_indlev = train['train']['__index_level_0__'][:]
temp_indlev.extend(ppg['__index_level_0__'].to_list())
len(temp_indlev)

7904

In [ ]:
temp_ans = train['train']['answers'][:]
temp_ans.extend(ppg['answers'].to_list())
len(temp_ans)

In [33]:
train['train']['id'] = temp_id

TypeError: 'Dataset' object does not support item assignment

In [5]:
train['train']['id'].extend(ppg['id'].to_list())
len(train['train']['id'])

3952

In [7]:
count = 0
for mrc_id in train['train']['id']:
    count += 1
    
print(count)

3952


In [9]:
train['train']['question'].extend(ppg['question'].to_list())
# len(train['train']['question'])

In [16]:
len(train['train']['id'])

3952

In [22]:
# a = [1, 2, 3]
a.extend([4])

In [23]:
a

[1, 2, 3, 4, 4]